In [195]:
from utils_filter import *
from utils_datetime import *
from dateutil import parser
from dateutil.relativedelta import relativedelta

Subsets outlooks, pph, and report data in various ways (add label to each datapoint for each method of subsetting so can be pulled out of full dataset later)


In [91]:
data_location = 'data'
moderate = False # only consider moderate days
outlooks, pph, reports = read_datasets(data_location, moderate)

reading outlooks 1
reading outlooks 2


c:\Users\miles\OneDrive\Documents\UW\Research\utils_filter.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outlooks = outlooks.append(gp.read_file(data_location + '/outlooks/' + mod_string + '_outlooks_2.shp'))


reading pph
reading storm reports


In [196]:
tz_conversions = {'PST': timedelta(hours=8),
                  'MST': timedelta(hours=7),
                  'CST': timedelta(hours=6),
                  'CSt': timedelta(hours=6),
                  'CSC': timedelta(hours=6),
                  'SCT': timedelta(hours=6),
                  'EST': timedelta(hours=5),
                  'ESt': timedelta(hours=5),
                  'PDT': timedelta(hours=7),
                  'MDT': timedelta(hours=6),
                  'CDT': timedelta(hours=5),
                  'EDT': timedelta(hours=4),
                  'HST': timedelta(hours=10),
                  'SST': timedelta(hours=11),
                  'GST': timedelta(hours=10),
                  'AKS': timedelta(hours=9),
                  'AST': timedelta(hours=4),
                  'UNK': timedelta(hours=5),
                  'GMT': timedelta(0)}

def get_reports_date_strings(date_times, timezones):
    # returns list of strings of date of given datetime and timezone (where day cutoffs are 12-12 UTC) formatted as 'YYYYMMDD0000'
    for datetime, timezone, i in zip(date_times, timezones, range(len(timezones))):
        #print(datetime + ' ' + timezone[:3])
        datetime = parser.parse(datetime)
        datetime = datetime + tz_conversions[timezone[:3]]
        #print(datetime)
        if (datetime.hour < 12):
            datetime = datetime - timedelta(days = 1)
        if datetime.year > 2049:
            datetime = datetime - relativedelta(years = 100)
        datetime = datetime.strftime("%Y%m%d") + '0000'
        if i == 0:
            ret = [datetime]
        else:
            ret.append(datetime)
    return ret

def get_pph_date_strings(times):
    # returns a list of strings of given dates formatted as 'YYYYMMDD0000'
    for datetime, i in zip(times, range(len(times))):
        string = datetime.dt.strftime("%Y%m%d").values + '0000'
        if i == 0:
            ret = [string]
        else:
            ret.append(string)
    return ret

In [198]:
reports['DATE'] = get_reports_date_strings(reports['BEGIN_DATE_TIME'], reports['CZ_TIMEZONE']) #TODO: or end time?
pph['time'] = get_pph_date_strings(pph.time) 

TypeError: '.dt' accessor only available for DataArray with datetime64 timedelta64 dtype or for arrays containing cftime datetime objects.

In [ ]:
# TODO: subset outlooks into only one day 1, two day 2, and one day 3 categorical outlooks

In [50]:
def add_outlooks_label(outlooks, label_dates, labels, label_name, none_label):
    # adds new column with values from labels on the corresponding list of dates in label_dates (DONE)
    outlooks = outlooks
    outlooks[label_name] = none_label
    for label, dates in zip(labels, label_dates):
        #print(label)
        outlooks[label_name].loc[outlooks['DATE'].isin(dates)] = label
    return outlooks

def add_pph_label(pph, label_dates, labels, label_name, none_label):
    # TODO: add data variable named label_name with correct labels with only time as coordinate
    print("adding new variable in pph")
    pph[label_name] = (('time'), np.full(len(pph['time']), none_label))
    for label, dates in zip(labels, label_dates):
        print(label)
    #    new_pph[label_name].loc[new_pph['time'].isin(dates)] = label # TODO: uncomment once report dates come in correct format 
    return pph

def add_reports_label(reports, label_dates, labels, label_name, none_label):
    # TODO: verify that date column handles time zones correctly
    # TODO: add column with correct labels
    # new_reports[label_name] = none_label
    # for label, dates in zip(labels, label_dates):
    #    #print(label)
    #    new_reports[label_name].loc[new_reports['DATE'].isin(dates)] = label # TODO: syntax may be wrong
    return reports

def add_labels(outlooks, pph, reports, label_dates, labels, label_name, none_label):
    # adds labels, overwriting with later ones if a date has multiple labels
    return(add_outlooks_label(outlooks, label_dates, labels, label_name, none_label), 
           add_pph_label(pph, label_dates, labels, label_name, none_label),
           add_reports_label(reports, label_dates, labels, label_name, none_label))



In [204]:
pph

<xarray.Dataset>
Dimensions:             (time: 14975, x: 93, y: 65)
Coordinates:
  * time                (time) <U12 '197901010000' ... '201912310000'
  * x                   (x) float64 0.0 1.0 2.0 3.0 4.0 ... 89.0 90.0 91.0 92.0
  * y                   (y) float64 0.0 1.0 2.0 3.0 4.0 ... 61.0 62.0 63.0 64.0
Data variables:
    lat                 (y, x) float64 ...
    lon                 (y, x) float64 ...
    p_perfect_wind      (time, y, x) float64 ...
    p_perfect_sig_wind  (time, y, x) float64 ...
    p_perfect_hail      (time, y, x) float64 ...
    p_perfect_sig_hail  (time, y, x) float64 ...
    p_perfect_tor       (time, y, x) float64 ...
    p_perfect_sig_tor   (time, y, x) float64 ...
Attributes:
    title:         Practically Perfect Wind Hindcasts
    sigma:         1.5
    grid:          80-km NCEP 211
    author:        Dr. Victor Gensini
    author_email:  vgensini@niu.edu
    citation:      https://doi.org/10.1175/BAMS-D-19-0321.1

Subset by threshold

In [51]:
# add max threshold forecasted for valid day to each datapoint

categories = ['TSTM', 'MRGL', 'SLGT', 'ENH', 'MDT', 'HIGH']
category_dates = []
for category in categories:
    category_dates.append(identify_dates_above_threshold(outlooks, category))

(new_outlooks, new_pph, new_reports) = add_labels(outlooks, pph, reports, category_dates, categories, 'Categorical Risk', 'NONE')
new_pph


C:\Users\miles\AppData\Local\Temp\ipykernel_1080\1054700370.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outlooks[label_name].loc[outlooks['DATE'].isin(dates)] = label


adding new variable in pph
TSTM
MRGL
SLGT
ENH
MDT
HIGH


<xarray.Dataset>
Dimensions:             (time: 14975, x: 93, y: 65)
Coordinates:
  * time                (time) datetime64[ns] 1979-01-01T12:00:00 ... 2019-12...
  * x                   (x) float64 0.0 1.0 2.0 3.0 4.0 ... 89.0 90.0 91.0 92.0
  * y                   (y) float64 0.0 1.0 2.0 3.0 4.0 ... 61.0 62.0 63.0 64.0
Data variables:
    lat                 (y, x) float64 ...
    lon                 (y, x) float64 ...
    p_perfect_wind      (time, y, x) float64 ...
    p_perfect_sig_wind  (time, y, x) float64 ...
    p_perfect_hail      (time, y, x) float64 ...
    p_perfect_sig_hail  (time, y, x) float64 ...
    p_perfect_tor       (time, y, x) float64 ...
    p_perfect_sig_tor   (time, y, x) float64 ...
    Categorical Risk    (time) <U4 'NONE' 'NONE' 'NONE' ... 'NONE' 'NONE' 'NONE'
Attributes:
    title:         Practically Perfect Wind Hindcasts
    sigma:         1.5
    grid:          80-km NCEP 211
    author:        Dr. Victor Gensini
    author_email:  vgensini@niu.edu
    citation:      https://doi.org/10.1175/BAMS-D-19-0321.1

Subset by ramp up/down

In [ ]:
# define and add ramp category for each datapoint. Potentially add 2 binary ramp up and ramp down (4 options are [up, down, up and down, niether]). How many forecasts to consider for each day? The day 3, both day 2, and the first day 1 (so 4 forecasts ramp)

Subset by accurate/inaccurate

Consider forecast issue time for day 1--subset reports (and revise pph?)--look at other studies to see how they handled multiple day 1 outlooks (see the one in slack). Starting point should be use first forecasts, later ones would be a different analysis for another time. Decide if we should denote these somehow..

Subset by season

In [71]:
# add column denoting season (4 met seasons as starting point)

'198701010000'

Subset by region

In [ ]:
# decide how to do... lower priotity, could do by state of center of highest category (or center of PPH?) Could start with super basic version

Subset by environmental data (to do later)

In [ ]:
# resave labelled data
# TODO: create master function that subsets data based on easy inputs once it's all been labelled. Put in a util file